# 3. ANÁLISIS PREDICTIVO.

## 3.2 ¿Cuál es la mejor hora del día para ir al aeropuerto?

**1. Importar librerías**

In [0]:
import pandas as pd
import numpy as np

**2. Configuración para trabajar en el entorno de collaboratory**

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
from google.colab import files

In [0]:
csv_import = drive.CreateFile({'id':'1TlBINWeLJhWvVMqkeDW-jflVKEH3zv3A'})

In [0]:
 csv_import.GetContentFile('yellow_tripdata_2015-03.csv') 

**3. Leer datos**

In [0]:
dy3 = pd.read_csv('yellow_tripdata_2015-03.csv', nrows = 1000000)

**4. Visualización de los datos y shape de los mismos**

In [8]:
dy3.shape

(1000000, 19)

In [9]:
dy3.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2015-03-06 08:02:31,2015-03-06 08:09:55,1,1.2,-73.990211,40.750969,1,N,-73.987892,40.738037,2,7.0,0.0,0.5,0.00,0.0,0.3,7.80
1,1,2015-03-06 08:02:31,2015-03-06 08:15:23,1,3.2,-73.935188,40.800720,1,N,-73.952553,40.765373,2,11.5,0.0,0.5,0.00,0.0,0.3,12.30
2,1,2015-03-06 08:02:31,2015-03-06 08:12:27,1,1.1,-73.963753,40.767937,1,N,-73.956947,40.780270,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80
3,1,2015-03-06 08:02:31,2015-03-06 08:09:09,1,0.8,-73.997177,40.742168,1,N,-74.008064,40.739281,1,6.0,0.0,0.5,1.00,0.0,0.3,7.80
4,1,2015-03-06 08:02:32,2015-03-06 08:19:37,1,2.7,-74.006844,40.730267,1,N,-73.976860,40.750671,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55


**5. Limpieza de datos**

In [0]:
dy3 = dy3.drop(dy3[dy3['total_amount']<=0].index)
dy3 = dy3.drop(dy3[dy3['total_amount']<=1].index)
dy3 = dy3.drop(dy3[dy3['total_amount']>=1000].index)

In [0]:
dy3 = dy3.drop(dy3[dy3['pickup_latitude']<40.49].index)
dy3 = dy3.drop(dy3[dy3['pickup_latitude']>40.92].index)
dy3 = dy3.drop(dy3[dy3['pickup_longitude']<-74.27].index)
dy3 = dy3.drop(dy3[dy3['pickup_longitude']>-73.68].index)
dy3 = dy3.drop(dy3[dy3['dropoff_latitude']<40.49].index)
dy3 = dy3.drop(dy3[dy3['dropoff_latitude']>40.92].index)
dy3 = dy3.drop(dy3[dy3['dropoff_longitude']<-74.27].index)
dy3 = dy3.drop(dy3[dy3['dropoff_longitude']>-73.68].index)
dy3 = dy3.drop(dy3[dy3['trip_distance']<0.1].index)

**6. Importar librerías para cambiar formato de fechas**

In [0]:
from datetime import datetime,date
import time

**7. Crear dos nuevas variables, duraciñon del trayecto en segundos y minutos**

In [0]:
dy3['Time_carreer_seconds'] = dy3.apply(
    lambda s:
    time.mktime((datetime.strptime(s['tpep_dropoff_datetime'],"%Y-%m-%d %H:%M:%S")).timetuple())
    -time.mktime((datetime.strptime(s['tpep_pickup_datetime'],"%Y-%m-%d %H:%M:%S")).timetuple())
    ,axis=1)

In [0]:
dy3['Duración_del_trayecto_min'] = dy3.apply(
    lambda s:
    (s['Time_carreer_seconds'] % 3600) // 60
    ,axis=1)

**8. A  partir de las columnas de fechas, creamos variables que creemos que aportarán valor a nuestro modelo**

In [17]:
def add_datetime_features(dy3):
    
    year = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).year
    dy3['year'] = dy3['tpep_pickup_datetime'].map(year)
    print('1/7')
    
    hour = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).hour
    dy3['hour'] = dy3['tpep_pickup_datetime'].map(hour)
    print('2/7')

    day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).weekday()
    dy3['day_of_week'] = dy3['tpep_pickup_datetime'].map(day_of_week)
    print('3/7')

    month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month
    dy3['month'] = dy3['tpep_pickup_datetime'].map(month)
    print('4/7')

    week_number = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).strftime('%V')
    dy3['week_number'] = dy3['tpep_pickup_datetime'].map(week_number)
    print('5/7')
    
    seasons = [0,0,1,1,1,2,2,2,3,3,3,0] #dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).month-1)]
    dy3['season'] = dy3['tpep_pickup_datetime'].map(season)
    print('6/7')
    
    late_night_hours = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
    late_night = lambda x: late_night_hours[(datetime.strptime(x, "%Y-%m-%d %H:%M:%S" ).hour)]
    dy3['late_night'] = dy3['tpep_pickup_datetime'].map(late_night)
    print('7/7. All done!')
    
add_datetime_features(dy3)

1/7
2/7
3/7
4/7
5/7
6/7
7/7. All done!


**9. Creamos variable velocidad en millas / min**

In [0]:
dy3['velocidad'] = (dy3['trip_distance']/dy3['Duración_del_trayecto_min'])

In [29]:
dy3.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Time_carreer_seconds,Duración_del_trayecto_min,year,hour,day_of_week,month,week_number,season,late_night,velocidad,JFKDist,EWRDist,LGADist
0,1,2015-03-06 08:02:31,2015-03-06 08:09:55,1,1.2,-73.990211,40.750969,1,N,-73.987892,40.738037,2,7.0,0.0,0.5,0.00,0.0,0.3,7.80,444.0,7.0,2015,8,4,3,10,1,0,0.171429,20.737599,16.050834,10.326384
1,1,2015-03-06 08:02:31,2015-03-06 08:15:23,1,3.2,-73.935188,40.800720,1,N,-73.952553,40.765373,2,11.5,0.0,0.5,0.00,0.0,0.3,12.30,772.0,12.0,2015,8,4,3,10,1,0,0.266667,20.236671,19.927789,5.879235
2,1,2015-03-06 08:02:31,2015-03-06 08:12:27,1,1.1,-73.963753,40.767937,1,N,-73.956947,40.780270,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80,596.0,9.0,2015,8,4,3,10,1,0,0.122222,21.120942,19.192002,7.109131
3,1,2015-03-06 08:02:31,2015-03-06 08:09:09,1,0.8,-73.997177,40.742168,1,N,-74.008064,40.739281,1,6.0,0.0,0.5,1.00,0.0,0.3,7.80,398.0,6.0,2015,8,4,3,10,1,0,0.133333,21.644540,14.496399,11.193321
4,1,2015-03-06 08:02:32,2015-03-06 08:19:37,1,2.7,-74.006844,40.730267,1,N,-73.976860,40.750671,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55,1025.0,17.0,2015,8,4,3,10,1,0,0.158824,20.754939,14.266830,9.263893


**10. Creamos la distancia esférica para el posterior cálculo del punto 11.**

In [0]:
def sphere_dist(pickup_lat, pickup_lon, dropoff_lat, dropoff_lon):
   
    R_earth = 6371
    pickup_lat, pickup_lon, dropoff_lat, dropoff_lon = map(np.radians,
                                                             [pickup_lat, pickup_lon, 
                                                              dropoff_lat, dropoff_lon])
    dlat = dropoff_lat - pickup_lat
    dlon = dropoff_lon - pickup_lon
    
    a = np.sin(dlat/2.0)**2 + np.cos(pickup_lat) * np.cos(dropoff_lat) * np.sin(dlon/2.0)**2
    
    return 2 * R_earth * np.arcsin(np.sqrt(a))

**11. Cálculo de las distancias a los aeropuertos**

In [0]:
def add_airport_dist(dataset):
    """
    JFK: John F. Kennedy International Airport
    EWR: Newark Liberty International Airport
    LGA: LaGuardia Airport
    """
    jfk_coord = (40.639722, -73.778889)
    ewr_coord = (40.6925, -74.168611)
    lga_coord = (40.77725, -73.872611)
    
    pickup_lat = dataset['pickup_latitude']
    dropoff_lat = dataset['dropoff_latitude']
    pickup_lon = dataset['pickup_longitude']
    dropoff_lon = dataset['dropoff_longitude']
    
    pickup_jfk = sphere_dist(pickup_lat, pickup_lon, jfk_coord[0], jfk_coord[1]) 
    dropoff_jfk = sphere_dist(jfk_coord[0], jfk_coord[1], dropoff_lat, dropoff_lon) 
    pickup_ewr = sphere_dist(pickup_lat, pickup_lon, ewr_coord[0], ewr_coord[1])
    dropoff_ewr = sphere_dist(ewr_coord[0], ewr_coord[1], dropoff_lat, dropoff_lon) 
    pickup_lga = sphere_dist(pickup_lat, pickup_lon, lga_coord[0], lga_coord[1]) 
    dropoff_lga = sphere_dist(lga_coord[0], lga_coord[1], dropoff_lat, dropoff_lon) 
    
    dataset['JFKDist'] = pd.concat([pickup_jfk, dropoff_jfk], axis=1).min(axis=1)
    dataset['EWRDist'] = pd.concat([pickup_ewr, dropoff_ewr], axis=1).min(axis=1)
    dataset['LGADist'] = pd.concat([pickup_lga, dropoff_lga], axis=1).min(axis=1)
    
    return dataset

**12. Añadimos el dataset creado al dataset existente**

In [0]:
dy3 = add_airport_dist(dy3)

**13. Visualización de los datos**

In [27]:
dy3.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Time_carreer_seconds,Duración_del_trayecto_min,year,hour,day_of_week,month,week_number,season,late_night,velocidad,JFKDist,EWRDist,LGADist
0,1,2015-03-06 08:02:31,2015-03-06 08:09:55,1,1.2,-73.990211,40.750969,1,N,-73.987892,40.738037,2,7.0,0.0,0.5,0.00,0.0,0.3,7.80,444.0,7.0,2015,8,4,3,10,1,0,10.285714,20.737599,16.050834,10.326384
1,1,2015-03-06 08:02:31,2015-03-06 08:15:23,1,3.2,-73.935188,40.800720,1,N,-73.952553,40.765373,2,11.5,0.0,0.5,0.00,0.0,0.3,12.30,772.0,12.0,2015,8,4,3,10,1,0,16.000000,20.236671,19.927789,5.879235
2,1,2015-03-06 08:02:31,2015-03-06 08:12:27,1,1.1,-73.963753,40.767937,1,N,-73.956947,40.780270,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80,596.0,9.0,2015,8,4,3,10,1,0,7.333333,21.120942,19.192002,7.109131
3,1,2015-03-06 08:02:31,2015-03-06 08:09:09,1,0.8,-73.997177,40.742168,1,N,-74.008064,40.739281,1,6.0,0.0,0.5,1.00,0.0,0.3,7.80,398.0,6.0,2015,8,4,3,10,1,0,8.000000,21.644540,14.496399,11.193321
4,1,2015-03-06 08:02:32,2015-03-06 08:19:37,1,2.7,-74.006844,40.730267,1,N,-73.976860,40.750671,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55,1025.0,17.0,2015,8,4,3,10,1,0,9.529412,20.754939,14.266830,9.263893


**14. Creación variable tiempo a aeropuerto**

In [0]:
dy3['Tiempo_aeropuerto_JFK'] = dy3['JFKDist'] / dy3['velocidad']
dy3['Tiempo_aeropuerto_EWR'] = dy3['EWRDist'] / dy3['velocidad']
dy3['Tiempo_aeropuerto_LGA'] = dy3['LGADist'] / dy3['velocidad']

**15. Visualización datos**

In [31]:
dy3.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Time_carreer_seconds,Duración_del_trayecto_min,year,hour,day_of_week,month,week_number,season,late_night,velocidad,JFKDist,EWRDist,LGADist,Tiempo_aeropuerto_JFK,Tiempo_aeropuerto_EWR,Tiempo_aeropuerto_LGA
0,1,2015-03-06 08:02:31,2015-03-06 08:09:55,1,1.2,-73.990211,40.750969,1,N,-73.987892,40.738037,2,7.0,0.0,0.5,0.00,0.0,0.3,7.80,444.0,7.0,2015,8,4,3,10,1,0,0.171429,20.737599,16.050834,10.326384,120.969328,93.629866,60.237240
1,1,2015-03-06 08:02:31,2015-03-06 08:15:23,1,3.2,-73.935188,40.800720,1,N,-73.952553,40.765373,2,11.5,0.0,0.5,0.00,0.0,0.3,12.30,772.0,12.0,2015,8,4,3,10,1,0,0.266667,20.236671,19.927789,5.879235,75.887515,74.729209,22.047133
2,1,2015-03-06 08:02:31,2015-03-06 08:12:27,1,1.1,-73.963753,40.767937,1,N,-73.956947,40.780270,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80,596.0,9.0,2015,8,4,3,10,1,0,0.122222,21.120942,19.192002,7.109131,172.807707,157.025471,58.165619
3,1,2015-03-06 08:02:31,2015-03-06 08:09:09,1,0.8,-73.997177,40.742168,1,N,-74.008064,40.739281,1,6.0,0.0,0.5,1.00,0.0,0.3,7.80,398.0,6.0,2015,8,4,3,10,1,0,0.133333,21.644540,14.496399,11.193321,162.334048,108.722992,83.949909
4,1,2015-03-06 08:02:32,2015-03-06 08:19:37,1,2.7,-74.006844,40.730267,1,N,-73.976860,40.750671,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55,1025.0,17.0,2015,8,4,3,10,1,0,0.158824,20.754939,14.266830,9.263893,130.679245,89.828187,58.328213


**16. Describe de variable ejemplo tiempo aeropuerto a JFK**

In [36]:
dy3['Tiempo_aeropuerto_JFK'].describe()

count    970928.000000
mean        123.467030
std          81.803244
min           0.000021
25%          78.205877
50%         110.132473
75%         151.560135
max        9699.294485
Name: Tiempo_aeropuerto_JFK, dtype: float64

**17. Análisis quantiles de variables creadas**

In [39]:
quantile_Tiempo_aeropuerto_JFK = dy3.Tiempo_aeropuerto_JFK.quantile(np.round(np.arange(0.00, 1.01, 0.01), 2))
qValues = np.round(np.arange(0.01, 1.01, 0.01), 2)
for i in qValues:
    print("{} Valor del percentil tiempo a JFK: {}min".format((int(i*100)), quantile_Tiempo_aeropuerto_JFK[i]))

1 Valor del percentil tiempo a JFK: 1.4192504706221512min
2 Valor del percentil tiempo a JFK: 2.4397139674726214min
3 Valor del percentil tiempo a JFK: 24.315781241503796min
4 Valor del percentil tiempo a JFK: 33.03977234888428min
5 Valor del percentil tiempo a JFK: 37.628481459256854min
6 Valor del percentil tiempo a JFK: 41.41934119718066min
7 Valor del percentil tiempo a JFK: 44.54689412413833min
8 Valor del percentil tiempo a JFK: 47.402623260714755min
9 Valor del percentil tiempo a JFK: 50.05582062685096min
10 Valor del percentil tiempo a JFK: 52.45439181153034min
11 Valor del percentil tiempo a JFK: 54.64764450026787min
12 Valor del percentil tiempo a JFK: 56.7758372415613min
13 Valor del percentil tiempo a JFK: 58.771400388563876min
14 Valor del percentil tiempo a JFK: 60.71603078750697min
15 Valor del percentil tiempo a JFK: 62.54147803838875min
16 Valor del percentil tiempo a JFK: 64.35979704898809min
17 Valor del percentil tiempo a JFK: 66.12979137704252min
18 Valor del perce

In [51]:
quantile_Tiempo_aeropuerto_EWR = dy3.Tiempo_aeropuerto_EWR.quantile(np.round(np.arange(0.00, 1.01, 0.01), 2))
qValues = np.round(np.arange(0.01, 1.01, 0.01), 2)
for i in qValues:
    print("{} Valor del percentil tiempo a EWR: {}min".format((int(i*100)), quantile_Tiempo_aeropuerto_EWR[i]))

1 Valor del percentil tiempo a EWR: 27.77840875791388min
2 Valor del percentil tiempo a EWR: 32.0703328505247min
3 Valor del percentil tiempo a EWR: 35.08298592957436min
4 Valor del percentil tiempo a EWR: 37.621147310644595min
5 Valor del percentil tiempo a EWR: 39.77830656923471min
6 Valor del percentil tiempo a EWR: 41.702804427900645min
7 Valor del percentil tiempo a EWR: 43.4830921885357min
8 Valor del percentil tiempo a EWR: 45.076078247454674min
9 Valor del percentil tiempo a EWR: 46.63684122421472min
10 Valor del percentil tiempo a EWR: 48.08672450487106min
11 Valor del percentil tiempo a EWR: 49.44356868183631min
12 Valor del percentil tiempo a EWR: 50.71839975566026min
13 Valor del percentil tiempo a EWR: 51.94297089486116min
14 Valor del percentil tiempo a EWR: 53.14146058755354min
15 Valor del percentil tiempo a EWR: 54.31099240753559min
16 Valor del percentil tiempo a EWR: 55.44882397832703min
17 Valor del percentil tiempo a EWR: 56.56902398878813min
18 Valor del percentil

In [52]:
quantile_Tiempo_aeropuerto_LGA = dy3.Tiempo_aeropuerto_LGA.quantile(np.round(np.arange(0.00, 1.01, 0.01), 2))
qValues = np.round(np.arange(0.01, 1.01, 0.01), 2)
for i in qValues:
    print("{} Valor del percentil tiempo a LGA: {}min".format((int(i*100)), quantile_Tiempo_aeropuerto_LGA[i]))

1 Valor del percentil tiempo a LGA: 1.0417912670187859min
2 Valor del percentil tiempo a LGA: 2.0793769164827816min
3 Valor del percentil tiempo a LGA: 3.5990385746196636min
4 Valor del percentil tiempo a LGA: 9.689308578736696min
5 Valor del percentil tiempo a LGA: 13.573498123711957min
6 Valor del percentil tiempo a LGA: 15.685824570766519min
7 Valor del percentil tiempo a LGA: 17.29678660947077min
8 Valor del percentil tiempo a LGA: 18.67325994805268min
9 Valor del percentil tiempo a LGA: 19.90675424134683min
10 Valor del percentil tiempo a LGA: 21.03893658491112min
11 Valor del percentil tiempo a LGA: 22.07711601070988min
12 Valor del percentil tiempo a LGA: 23.06495545864521min
13 Valor del percentil tiempo a LGA: 23.99308456554846min
14 Valor del percentil tiempo a LGA: 24.874136779981335min
15 Valor del percentil tiempo a LGA: 25.710148141330038min
16 Valor del percentil tiempo a LGA: 26.504670755412015min
17 Valor del percentil tiempo a LGA: 27.26481322128736min
18 Valor del pe

**18. Limpieza de datos erroneos bajo criterio porpio**

In [93]:
dy3['Tiempo_aeropuerto_JFK'].describe()

count    922551.000000
mean        124.098679
std          61.155190
min           2.430029
25%          82.191990
50%         112.446438
75%         152.572782
max         399.990063
Name: Tiempo_aeropuerto_JFK, dtype: float64

In [94]:
dy3['Tiempo_aeropuerto_EWR'].describe()

count    922551.000000
mean         99.738133
std          48.621289
min           0.200236
25%          66.001915
50%          89.364484
75%         121.527867
max         396.551191
Name: Tiempo_aeropuerto_EWR, dtype: float64

In [95]:
dy3['Tiempo_aeropuerto_LGA'].describe()

count    922551.000000
mean         54.164684
std          29.365304
min           2.430101
25%          33.728983
50%          48.494146
75%          68.532465
max         311.542916
Name: Tiempo_aeropuerto_LGA, dtype: float64

**19. Limpiamos los datos en base a la media, así en el cálculo posterior se obtiene la mejor hora para el tiempo medio que se tarda a cada aeropuerto.**

In [0]:
dy3 = dy3.drop(dy3[dy3['Tiempo_aeropuerto_JFK']<=0].index)
dy3 = dy3.drop(dy3[dy3['Tiempo_aeropuerto_JFK']<=123].index)
dy3 = dy3.drop(dy3[dy3['Tiempo_aeropuerto_JFK']>124].index)
dy3 = dy3.drop(dy3[dy3['Tiempo_aeropuerto_EWR']<=0].index)
dy3 = dy3.drop(dy3[dy3['Tiempo_aeropuerto_EWR']<=98].index)
dy3 = dy3.drop(dy3[dy3['Tiempo_aeropuerto_EWR']>100].index)
dy3 = dy3.drop(dy3[dy3['Tiempo_aeropuerto_LGA']<=0].index)
dy3 = dy3.drop(dy3[dy3['Tiempo_aeropuerto_LGA']<=53].index)
dy3 = dy3.drop(dy3[dy3['Tiempo_aeropuerto_LGA']>54].index)

In [0]:
best_hour_JFK = dy3[dy3['Tiempo_aeropuerto_JFK'] ==  np.min(dy3['Tiempo_aeropuerto_JFK'])]
best_hour_EWR = dy3[dy3['Tiempo_aeropuerto_EWR'] ==  np.min(dy3['Tiempo_aeropuerto_EWR'])]
best_hour_LGA = dy3[dy3['Tiempo_aeropuerto_LGA'] ==  np.min(dy3['Tiempo_aeropuerto_LGA'])]

**20. Resultados de mejor hora para viajar al aeropuerto**

In [98]:
print(best_hour_JFK['hour'])
print(best_hour_JFK['Tiempo_aeropuerto_JFK'])
print(best_hour_JFK['trip_distance'])
print(best_hour_EWR['hour'])
print(best_hour_EWR['Tiempo_aeropuerto_EWR'])
print(best_hour_EWR['trip_distance'])
print(best_hour_LGA['hour'])
print(best_hour_LGA['Tiempo_aeropuerto_LGA'])
print(best_hour_LGA['trip_distance'])

902396    14
Name: hour, dtype: int64
902396    123.053722
Name: Tiempo_aeropuerto_JFK, dtype: float64
902396    1.83
Name: trip_distance, dtype: float64
488522    22
Name: hour, dtype: int64
488522    98.236307
Name: Tiempo_aeropuerto_EWR, dtype: float64
488522    1.33
Name: trip_distance, dtype: float64
775871    21
Name: hour, dtype: int64
775871    53.228888
Name: Tiempo_aeropuerto_LGA, dtype: float64
775871    1.0
Name: trip_distance, dtype: float64
